# Chapter 3: Introduction to Data Analysis in Python Polars 

## Inspecting a DataFrame

### How to do it...

In [235]:
import polars as pl

In [236]:
df = pl.read_csv('../data/covid_19_dealths.csv')

In [237]:
df.head(3)

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null


In [238]:
df.tail(5)

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""",…,14,null,0,14,"""One or more da…"
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""",…,10,0,0,10,null
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""",…,null,null,0,null,"""One or more da…"
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""",…,36,null,0,38,"""One or more da…"
"""09/27/2023""","""09/01/2023""","""09/23/2023""","""By Month""","""2023""",…,42,null,null,44,"""One or more da…"


In [239]:
df.glimpse(max_items_per_column=3)

Rows: 137700
Columns: 16
$ Data As Of                               <str> '09/27/2023', '09/27/2023', '09/27/2023'
$ Start Date                               <str> '01/01/2020', '01/01/2020', '01/01/2020'
$ End Date                                 <str> '09/23/2023', '09/23/2023', '09/23/2023'
$ Group                                    <str> 'By Total', 'By Total', 'By Total'
$ Year                                     <str> None, None, None
$ Month                                    <str> None, None, None
$ State                                    <str> 'United States', 'United States', 'United States'
$ Sex                                      <str> 'All Sexes', 'All Sexes', 'All Sexes'
$ Age Group                                <str> 'All Ages', 'Under 1 year', '0-17 years'
$ COVID-19 Deaths                          <i64> 1146774, 519, 1696
$ Total Deaths                             <i64> 12303399, 73213, 130970
$ Pneumonia Deaths                         <i64> 1162844, 1056, 2961
$ P

In [240]:
df.estimated_size('mb')

37.42431926727295

In [241]:
import polars.selectors as cs
df.select(cs.numeric()).describe()

describe,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths"
str,f64,f64,f64,f64,f64,f64
"""count""",137700.0,137700.0,137700.0,137700.0,137700.0,137700.0
"""null_count""",39430.0,19509.0,44864.0,36884.0,26688.0,44233.0
"""mean""",313.586547,2841.952585,336.597085,152.513411,5.002468,505.491778
"""std""",5992.341375,56201.384331,6126.573599,2980.886938,110.606691,9256.951591
"""min""",0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",0.0,43.0,0.0,0.0,0.0,0.0
"""50%""",0.0,153.0,18.0,0.0,0.0,25.0
"""75%""",50.0,657.0,74.0,21.0,0.0,107.0
"""max""",1.146774e6,1.2303399e7,1.162844e6,569264.0,22229.0,1.760095e6


In [242]:
df.null_count()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32
0,0,0,0,2754,…,44864,36884,26688,44233,39804


### There is more...

In [243]:
print(df.head())


shape: (5, 16)
┌────────┬────────┬────────┬────────┬──────┬───────┬───┬────────┬────────┬────────┬────────┬───────┐
│ Data   ┆ Start  ┆ End    ┆ Group  ┆ Year ┆ Month ┆ … ┆ Pneumo ┆ Pneumo ┆ Influe ┆ Pneumo ┆ Footn │
│ As Of  ┆ Date   ┆ Date   ┆ ---    ┆ ---  ┆ ---   ┆   ┆ nia    ┆ nia    ┆ nza    ┆ nia,   ┆ ote   │
│ ---    ┆ ---    ┆ ---    ┆ str    ┆ str  ┆ str   ┆   ┆ Deaths ┆ and    ┆ Deaths ┆ Influe ┆ ---   │
│ str    ┆ str    ┆ str    ┆        ┆      ┆       ┆   ┆ ---    ┆ COVID- ┆ ---    ┆ nza,   ┆ str   │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆ i64    ┆ 19     ┆ i64    ┆ or COV ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ Deaths ┆        ┆ ID-1…  ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ ---    ┆        ┆ ---    ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ i64    ┆        ┆ i64    ┆       │
╞════════╪════════╪════════╪════════╪══════╪═══════╪═══╪════════╪════════╪══

In [244]:
with pl.Config() as config:
    config.set_tbl_cols(11)
    print(df.head(2))

shape: (2, 16)
┌────────┬────────┬────────┬────────┬──────┬───────┬───┬────────┬────────┬────────┬────────┬───────┐
│ Data   ┆ Start  ┆ End    ┆ Group  ┆ Year ┆ Month ┆ … ┆ Pneumo ┆ Pneumo ┆ Influe ┆ Pneumo ┆ Footn │
│ As Of  ┆ Date   ┆ Date   ┆ ---    ┆ ---  ┆ ---   ┆   ┆ nia    ┆ nia    ┆ nza    ┆ nia,   ┆ ote   │
│ ---    ┆ ---    ┆ ---    ┆ str    ┆ str  ┆ str   ┆   ┆ Deaths ┆ and    ┆ Deaths ┆ Influe ┆ ---   │
│ str    ┆ str    ┆ str    ┆        ┆      ┆       ┆   ┆ ---    ┆ COVID- ┆ ---    ┆ nza,   ┆ str   │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆ i64    ┆ 19     ┆ i64    ┆ or COV ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ Deaths ┆        ┆ ID-1…  ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ ---    ┆        ┆ ---    ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ i64    ┆        ┆ i64    ┆       │
╞════════╪════════╪════════╪════════╪══════╪═══════╪═══╪════════╪════════╪══

In [245]:
pl.Config.set_tbl_cols(11)
print(df.head(2))

shape: (2, 16)
┌────────┬────────┬────────┬────────┬──────┬───────┬───┬────────┬────────┬────────┬────────┬───────┐
│ Data   ┆ Start  ┆ End    ┆ Group  ┆ Year ┆ Month ┆ … ┆ Pneumo ┆ Pneumo ┆ Influe ┆ Pneumo ┆ Footn │
│ As Of  ┆ Date   ┆ Date   ┆ ---    ┆ ---  ┆ ---   ┆   ┆ nia    ┆ nia    ┆ nza    ┆ nia,   ┆ ote   │
│ ---    ┆ ---    ┆ ---    ┆ str    ┆ str  ┆ str   ┆   ┆ Deaths ┆ and    ┆ Deaths ┆ Influe ┆ ---   │
│ str    ┆ str    ┆ str    ┆        ┆      ┆       ┆   ┆ ---    ┆ COVID- ┆ ---    ┆ nza,   ┆ str   │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆ i64    ┆ 19     ┆ i64    ┆ or COV ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ Deaths ┆        ┆ ID-1…  ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ ---    ┆        ┆ ---    ┆       │
│        ┆        ┆        ┆        ┆      ┆       ┆   ┆        ┆ i64    ┆        ┆ i64    ┆       │
╞════════╪════════╪════════╪════════╪══════╪═══════╪═══╪════════╪════════╪══

## Casting data types

### How to do it...

In [246]:
import polars as pl

In [247]:
df = pl.read_csv('../data/covid_19_dealths.csv')
df.head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,818,143,219,1390,null


In [248]:
df.with_columns(
        pl.col('Data As Of').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Start Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.to_date('%m/%d/%Y').alias('End Date 2'),
        pl.col('Year').cast(pl.Int64)
).head()


Data As Of,Start Date,End Date,Group,Year,…,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote,End Date 2
date,date,date,str,i64,…,i64,i64,i64,str,date
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,569264,22229,1760095,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,95,64,1541,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,424,509,4716,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,66,177,1079,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,143,219,1390,null,2023-09-23


In [249]:
updated_df = (
    df.with_columns(
        pl.col('Data As Of').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Start Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.to_date('%m/%d/%Y').alias('End Date 2'),
        pl.col('Year').cast(pl.Int64)
    )
)

In [250]:
lf = pl.scan_csv('../data/covid_19_dealths.csv')
lf.with_columns(
        pl.col('Data As Of').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('Start Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.strptime(pl.Date, '%m/%d/%Y'),
        pl.col('End Date').str.to_date('%m/%d/%Y').alias('End Date 2'),
        pl.col('Year').cast(pl.Int64)
).collect().head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote,End Date 2
date,date,date,str,i64,…,i64,i64,i64,str,date
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,569264,22229,1760095,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,95,64,1541,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,424,509,4716,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,66,177,1079,null,2023-09-23
2023-09-27,2020-01-01,2023-09-23,"""By Total""",null,…,143,219,1390,null,2023-09-23


## Finding and removing duplicates values 

### How to do it

In [251]:
import polars as pl

In [252]:
df = pl.read_csv('../data/covid_19_dealths.csv')
df.head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,818,143,219,1390,null


In [253]:
df.shape

(137700, 16)

In [254]:
df.is_duplicated().sum()

0

In [255]:
df.is_unique().sum()

137700

In [256]:
df.n_unique()

137700

In [257]:
df.select(pl.all().n_unique())

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32
1,45,45,3,5,…,3556,2533,493,4264,2


In [258]:
df.n_unique(subset=['Start Date', 'End Date'])

50

In [259]:
(
    df
    .unique(subset=['Start Date', 'End Date'], keep='first')
    .select(row_count=pl.count())
)

row_count
u32
50


In [260]:
rows_to_keep = df.select(['Year', 'COVID-19 Deaths']).is_unique()
rows_to_keep.sum()

3940

In [261]:
df.filter(rows_to_keep).shape

(3940, 16)

In [262]:
df.filter(rows_to_keep).head()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,…,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,…,818,143,219,1390,null


### There is more...

In [263]:
df.approx_n_unique()

Data As Of,Start Date,End Date,Group,Year,…,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32
1,45,45,3,5,…,3540,2523,492,4242,2


## Masking sensitive data

### How to do it...

In [9]:
import polars as pl

In [64]:
df = pl.read_csv('../data/covid_19_dealths.csv')
df.head()

Data As Of,Start Date,End Date,Group,Year,Month,State,Sex,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""All Ages""",1146774,12303399,1162844,569264,22229,1760095,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""Under 1 year""",519,73213,1056,95,64,1541,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""0-17 years""",1696,130970,2961,424,509,4716,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""1-4 years""",285,14299,692,66,177,1079,null
"""09/27/2023""","""01/01/2020""","""09/23/2023""","""By Total""",null,null,"""United States""","""All Sexes""","""5-14 years""",509,22008,818,143,219,1390,null


In [65]:
import random

def get_random_nums(num_list, length):
    random_nums = ''.join(str(n) for n in random.sample(num_list, length))
    return random_nums

In [66]:
fake_ssns = []
nums = [n for n in range(0, 10)]

for i in range(0, df.height):
    part_1 = get_random_nums(nums, 3)
    part_2 = get_random_nums(nums, 2)
    part_3 = get_random_nums(nums, 4)
    fake_ssn = part_1 + '-' + part_2 + '-' + part_3
    fake_ssns.append(fake_ssn)

fake_ssns_df = pl.DataFrame({'SSN': fake_ssns})
fake_ssns_df.head()

SSN
str
"""306-27-7316"""
"""348-59-8710"""
"""928-87-5329"""
"""345-49-0542"""
"""428-52-3075"""


In [67]:
df = pl.concat([df, fake_ssns_df], how='horizontal')

In [95]:
df.select(
    ('XXX-XX-XX' + pl.col('SSN').str.slice(9, 2)).alias('SSN Masked')
).head()

SSN Masked
str
"""XXX-XX-XX16"""
"""XXX-XX-XX10"""
"""XXX-XX-XX29"""
"""XXX-XX-XX42"""
"""XXX-XX-XX75"""


In [96]:
df.select(
    ('XXX-XX-XX' + pl.col('SSN').str.slice(9, 2)).alias('SSN Masked'),
    
).head()

SSN Masked
str
"""XXX-XX-XX16"""
"""XXX-XX-XX10"""
"""XXX-XX-XX29"""
"""XXX-XX-XX42"""
"""XXX-XX-XX75"""


In [97]:
df.select(
    pl.col('SSN').hash()
).head()

SSN
u64
13579232216623324611
1540581911894599224
14915679507062015913
3185350992463740989
8710751241438705090
